In [1]:
import plotly.express as px
import numpy as np

In [2]:
from neo4j import GraphDatabase
from tqdm.notebook import tqdm
import json

import pandas as pd

In [3]:
auth_path = './data/neo4j_graph/auth.json'
with open(auth_path, 'r') as f:
    auth = json.load(f)

# ローカルの場合は通常 uri: bolt(or neo4j)://localhost:7687, user: neo4j, pd: 設定したもの
# サンドボックスの場合は作成画面から接続情報が見られます
uri = 'neo4j://localhost:7687'
driver = GraphDatabase.driver(uri=uri, auth=(auth['user'], auth['pd']))

# Sandboxの場合はこんな感じ
# uri = 'bolt://54.175.38.249:35275'
# driver = GraphDatabase.driver(uri=uri, auth=('neo4j', 'spray-missile-sizing'))

In [20]:
with driver.session() as session:
    country_list = session.run('''
    MATCH (c:Country)
    WHERE exists(c.vector_epoch1) AND exists(c.vector_epoch2) AND exists(c.vector_epoch3)
    RETURN c.name as name
    ''')

In [21]:
country_list = pd.DataFrame([r.data() for r in country_list])

In [22]:
country_list

,name
0,Australia
1,Austria
2,Belgium
3,Cambodia
4,Canada
...,...
59,United Arab Emirates
60,United Kingdom
61,United States
62,Uruguay


In [33]:
country_name = 'Australia'

In [40]:
country_list['name'].to_list()

['Australia',
 'Austria',
 'Belgium',
 'Cambodia',
 'Canada',
 'Channel Islands',
 'China',
 'Ivory Coast',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Egypt',
 'Estonia',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Hong Kong',
 'India',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Jersey',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'South Korea',
 'Kuwait',
 'Latvia',
 'Lebanon',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Macao',
 'Malaysia',
 'Mauritius',
 'Mexico',
 'Moldova, Republic of',
 'Monaco',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Panama',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Russia',
 'Singapore',
 'Spain',
 'Sweden',
 'Switzerland',
 'Taiwan',
 'Tanzania, United Republic of',
 'Turkey',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'Uruguay',
 'Vietnam']

In [60]:
with driver.session() as session:
    country_vecs = session.run('''
    MATCH (c:Country {name: $country_name})-[r:TRANSFERRED_COUNTRY_SEPARATE]-(c_around)
    WHERE c_around.name in $country_list
    RETURN c_around.name as name, c_around.vector_epoch1 as vec1, c_around.vector_epoch2 as vec2, c_around.vector_epoch3 as vec3, count(r) AS transaction_count ORDER BY transaction_count DESC　LIMIT 15
    ''', country_name=country_name, country_list=country_list['name'].to_list())
    
country_vecs_list = [c.data() for c in country_vecs]
    
with driver.session() as session:
    country_vec = session.run('''
    MATCH (c:Country {name: $country_name})
    RETURN c.name as name, c.vector_epoch1 as vec1, c.vector_epoch2 as vec2, c.vector_epoch3 as vec3
    ''', country_name=country_name, country_list=country_list['name'].to_list())

country_vecs_list += [c.data() for c in country_vec]

country_vecs = pd.DataFrame(country_vecs_list)

for vec_type in ['vec1', 'vec2', 'vec3']:
    
    fig_vec = px.scatter(x=country_vecs['vec1'].apply(lambda x: x[0]), 
               y=country_vecs['vec2'].apply(lambda x: x[1]),
               text=country_vecs['name'],
               color=country_vecs['name'].apply(lambda x: {True: 'red', False: 'blue'}[x==country_name]),
               size=country_vecs['name'].apply(lambda x: {True: 20, False: 5}[x==country_name])
              )

In [28]:
with driver.session() as session:
    transaction_count = session.run('''
    MATCH (c:Country {name:$country_name})-[r:TRANSFERRED_COUNTRY_SEPARATE]-()
    RETURN r.begin.year AS year, count(r) AS transaction_count ORDER BY year 
    ''', country_name=country_name)

transaction_count = pd.DataFrame([r.data() for r in transaction_count])

transaction_count

,year,transaction_count
0,2011,1
1,2012,81
2,2013,1
3,2014,10
4,2015,15
5,2016,12


In [30]:
fig = px.line(transaction_count, x='year', y='transaction_count')